## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-08-20-56-49 +0000,nypost,Israeli fighter jets escort commercial flight ...,https://nypost.com/2026/02/08/world-news/israe...
1,2026-02-08-20-52-37 +0000,nyt,"7 Days, No Suspects: The Disappearance of Nanc...",https://www.nytimes.com/2026/02/08/us/nancy-gu...
2,2026-02-08-20-43-12 +0000,bbc,Japan's PM Takaichi on course for landslide vi...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
3,2026-02-08-20-40-04 +0000,wapo,Japan’s firebrand prime minister pulls off vic...,https://www.washingtonpost.com/world/2026/02/0...
4,2026-02-08-20-38-59 +0000,nyt,"Portugal Votes for President, With Leftist Set...",https://www.nytimes.com/2026/02/08/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2464/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,trump,17
142,super,13
143,bowl,13
23,election,12
16,japan,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
100,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...,49
18,2026-02-08-20-06-57 +0000,nypost,Super Bowl becomes who’s who of tech elite as ...,https://nypost.com/2026/02/08/us-news/silicon-...,48
159,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...,47
2,2026-02-08-20-43-12 +0000,bbc,Japan's PM Takaichi on course for landslide vi...,https://www.bbc.com/news/articles/cx2y7d2z29xo...,46
111,2026-02-08-10-50-00 +0000,startribune,Analysis: Super Bowl GMs show just how off-tar...,https://www.startribune.com/vikings-general-ma...,46


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
100,49,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...
18,48,2026-02-08-20-06-57 +0000,nypost,Super Bowl becomes who’s who of tech elite as ...,https://nypost.com/2026/02/08/us-news/silicon-...
2,46,2026-02-08-20-43-12 +0000,bbc,Japan's PM Takaichi on course for landslide vi...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
30,36,2026-02-08-19-23-55 +0000,nypost,Why Savannah Guthrie shouldn’t pay mom Nancy’s...,https://nypost.com/2026/02/08/us-news/why-sava...
58,35,2026-02-08-17-35-40 +0000,nypost,UK PM Keir Starmer’s top aide Morgan McSweeney...,https://nypost.com/2026/02/08/world-news/uk-pm...
159,30,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...
54,30,2026-02-08-18-04-45 +0000,nypost,Iran sentences locked-up Nobel Peace Prize win...,https://nypost.com/2026/02/08/world-news/iran-...
112,27,2026-02-08-10-30-00 +0000,wsj,The pace of hiring in the U.S. has dropped off...,https://www.wsj.com/economy/jobs/this-is-why-i...
129,26,2026-02-08-09-00-00 +0000,cbc,"No desks, no strategy: Experts say government'...",https://www.cbc.ca/news/canada/ottawa/no-desks...
43,25,2026-02-08-18-35-44 +0000,startribune,Live: Lindsey Vonn crashes in Olympic downhill...,https://www.startribune.com/2026-winter-olympi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
